In [1]:
from library_dicom.dicom_processor.tools.pyradiomic import *
import SimpleITK as sitk 
import os 
import numpy as np 
import csv

ModuleNotFoundError: No module named 'radiomics'

In [ ]:
#AHL NIFTI 
csv_path= '/media/oncopole/83c5223d-7a01-4ed0-b268-b877a7da96e2/AHL_NIFTI/AHL2011_NIFTI_V4.csv'
with open(csv_path, 'r') as csv_file :
    reader = csv.reader(csv_file, delimiter = ',') #liste pour chaque ligne 
    csv_file = []
    for row in reader :
        csv_file.append(row)
        
del csv_file[0] #enlever première ligne

#AHL NIFTI UNCONSTANT
csv_path_2= '/media/oncopole/DD 2To/AHL/NIFTI/AHL2011_NIFTI_unconstant.csv'
with open(csv_path_2, 'r') as csv_file :
    reader = csv.reader(csv_file, delimiter = ',') #liste pour chaque ligne 
    csv_file_2 = []
    for row in reader :
        csv_file_2.append(row)
        
del csv_file_2[0] #enlever première ligne

#### - Keep only pet0 

In [ ]:
pet0 = []
for row in csv_file : 
    if 'pet0' in row : 
        pet0.append(row)

#### - Rewrite abs path 

In [ ]:
base_path = '/media/oncopole/83c5223d-7a01-4ed0-b268-b877a7da96e2'
for row in pet0 : 
    ct = row[-3]
    pt = row[-2]
    mask = row[-1]
    up_ct = os.path.join(base_path, ct)
    up_pt = os.path.join(base_path, pt)
    up_mask = os.path.join(base_path, mask)
    row[-3] = up_ct 
    row[-2] = up_pt 
    row[-1] = up_mask

#### - Prepare dataset 

In [ ]:
dataset = []
for data in pet0 : 
    dataset.append(data)
for data in csv_file_2 : 
    dataset.append(data)

In [ ]:
#find date in json 
import json 
liste_json = []
list_dir = os.listdir('/media/deeplearning/Elements/AHL_JSON')
for file_ in list_dir : 
    liste_json.append(os.path.join('/media/deeplearning/Elements/AHL_JSON', file_))


for data in dataset : 
    for json_path in liste_json : 
        with open(json_path) as json_file : 
            reader = json.load(json_file)

            uid = reader['study']['StudyInstanceUID']

            if uid == data[2] : 

                data.append(reader['study']['StudyDate'])

                break

In [ ]:
for data in dataset : 
    year = data[-1][0:4]
    month = data[-1][4:6]
    day = data[-1][6:8]
    l = [year, month, day]
    date = "_".join(l)

    data.append(date)

In [ ]:
#check if every data has its date
cpt = 0
for data in dataset : 
    if len(data) != 9 : 
        cpt += 1

cpt

In [ ]:
distance_max = []
for nifti in dataset: 
    try : 
        print(dataset.index(nifti))
        liste_center = get_center_of_mass(nifti[-3], thresh = True, pet_path=nifti[-4])
        d_max = calcul_distance_max(liste_center)
        nifti.append(d_max)
    except Exception as err : 
        print(err)

In [ ]:
nifti_directory = '/home/deeplearning/AHL'
filename = 'AHL2011_dMax_v3.csv'

with open(os.path.join(nifti_directory, filename), 'w') as csv_file : 
    csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(["PATIENT ID", "DATE", "STUDY_UID", "D_MAX"])
    for row in good: 
        csv_writer.writerow([row[1], row[-2], row[2], row[-1]])